# Benchmarking foldseek ctranslate2 branch
Apply the [foldseek papaer](https://doi.org/10.1038/s41587-023-01773-0) benchmark using the `ctranslate2` branch.
The paper uses the [SCOPe](https://wwwuser.gwdg.de/~compbiol/foldseek/scop40pdb.tar.gz) 40% sequence redundance database.
However, we don't download the data here, but reuse existing `.pdb` data on the luster at `/home/sukhwan/foldseek-analysis/scope_pdb`.

In [ ]:
mkdir -p test/data/scope
ln -s /home/sukhwan/foldseek-analysis/scope_pdb test/data/scope/pdb

In [ ]:
# create foldseek db
srun -c 32 -t 20-0 --pty /bin/bash
DB_PATH=test/dbs/scope/
SCOPE_DATA=test/data/scope/pdb
mkdir -p $DB_PATH
# conda activate prostt5
# foldseek createdb --mask-bfactor-threshold -10000 $SCOPE_DATA $DB_PATH/scope_full
foldseek createdb $SCOPE_DATA $DB_PATH/scope_full

foldseek convert2fasta $DB_PATH/scope_full $(dirname $SCOPE_DATA)/scope_full.aa.fasta

In [ ]:
# create foldseek db from amino acid fasta using ProstT5
srun -p gpu --gres=gpu:1 -c 4 -t 20-0 --pty /bin/bash
MODEL_PATH=/home/sukhwan/foldseek_ctranslate/foldseek/weights/model 
cuda-build/src/foldseek createdb test/data/scope/scope_full.aa.fasta test/dbs/scope/scope_prostt5_gpu --prostt5-model $MODEL_PATH --gpu 1
# cpu-build/src/foldseek createdb test/data/scope/scope_full.aa.fasta test/dbs/scope/scope_prostt5 --prostt5-model $MODEL_PATH
exit

In [ ]:
# conda activate prostt5
srun -c 64 -t 20-0 --pty /bin/bash
DB_PATH=test/dbs/scope/
mkdir -p $DB_PATH/tmp

foldseek search $DB_PATH/scope_prostt5_gpu $DB_PATH/scope_full $DB_PATH/aln_scope_benchmark $DB_PATH/tmp/ -s 9.5 --max-seqs 2000 -e 10

foldseek convertalis $DB_PATH/scope_prostt5_gpu $DB_PATH/scope_full $DB_PATH/aln_scope_benchmark $DB_PATH/aln_scope_benchmark.m8

# alternatively clone git@github.com:steineggerlab/foldseek-analysis.git
AWK_SCRIPT=../../../consensus3Di/lib/foldseek-analysis/scopbenchmark/scripts/bench.noselfhit.awk
LOOKUP_FILE=../../../consensus3Di/lib/foldseek-analysis/scopbenchmark/data/scop_lookup.fix.tsv
$AWK_SCRIPT $LOOKUP_FILE <(cat $DB_PATH/aln_scope_benchmark.m8 | sed -E 's|.pdb||g') > test/data/benchmark.ctranslate2.splitting.rocx

awk '{ famsum+=$3; supfamsum+=$4; foldsum+=$5}END{print famsum/NR,supfamsum/NR,foldsum/NR}' test/data/benchmark.ctranslate2.splitting.rocx
# 0.841243 0.444564 0.0849547